# SMART
This file loads the trained category and type classifiers. The predicts and evaluate the results.

In [1]:
import utils
import pickle
import gensim 
from gensim.models import Word2Vec 

## Classify the category

In [2]:
clf_category = pickle.load(open('category_classifier.sav', 'rb'))

In [3]:
test = utils.load_dataset('../smart-dataset/datasets/Wikidata/lcquad2_anstype_wikidata_test_gold.json')

In [4]:
_, _, X_test, y_test = utils.prepare_X_y({}, test)
test_vectors = utils.transform_dataset(X_test, 'category_vectorizer.sav')

In [5]:
pred_category = clf_category.predict(test_vectors)
print("Accuracy:", sum(pred_category==y_test)/len(pred_category))

Accuracy: 0.9022095821483264


## Classify the type

In [6]:
clf_type_literal = pickle.load(open('type_literal_classifier.sav', 'rb'))
word2vec_sg = pickle.load(open('word2vec_sg.sav', 'rb'))

In [7]:
word2vec_sg.wv.most_similar('cola')

[('carbonated', 0.9915328025817871),
 ('non-alcoholic', 0.988882303237915),
 ('soft', 0.9813560843467712),
 ('drink', 0.9802319407463074),
 ('airliner', 0.973507285118103),
 ('equipment', 0.9714939594268799),
 ('beverage', 0.9703865051269531),
 ('fraction', 0.967510461807251),
 ('low-cost', 0.96715247631073),
 ('motorcycle', 0.9666259288787842)]

In [8]:
def split_bool_literal_reference(X, y):
    bool_map = {}
    literal_map = {}
    resource_map = {}

    for i in range(len(y)):
        if y[i] == 'boolean':
            bool_map[i] = X[i]
        elif y[i] == 'literal':
            literal_map[i] = X[i]
        elif y[i] == 'resource':
            resource_map[i] = X[i]
            
    return bool_map, literal_map, resource_map

In [9]:
bool_map, literal_map, resource_map = split_bool_literal_reference(X_test, pred_category)


In [10]:
   
X_test_literal = list(literal_map.values())
y_test_literal = []
for i in list(literal_map.keys()):
    y_test_literal.append(test[i]['type'][0])

In [11]:
test_vectors_literal = utils.transform_dataset(X_test_literal, 'literal_vectorizer.sav')

In [12]:
pred_literal = clf_type_literal.predict(test_vectors_literal)
print("Accuracy:", sum(pred_literal==y_test_literal)/len(pred_literal))

Accuracy: 0.8540596094552929
